In [ ]:
#meta 2/7/2021 Kaggle Cassava Module 2-Inference Mini (for rapid iteration)
#fast.ai 2020 study group Kaggle Competition
#src https://www.kaggle.com/c/cassava-leaf-disease-classification
#prev competition src https://www.kaggle.com/c/cassava-disease
#references https://arxiv.org/pdf/1908.02900.pdf

#input: export.pkl
#output: submission.csv
#pwd: /kaggle/working
#summary: Export a model and submit predictions on mini data.
#based on version of my-kaggle-cassava-2-infer.ipynb -> my-kaggle-cassava-2-infer_bs56_score0884.ipynb
#RESET: to version of my-kaggle-cassava-2-infer-mini.ipynb -> my-kaggle-cassava-2-infer-mini-res50_score0870.ipynb


#prev in my-kaggle-cassava-2-infer_bs56_score0884.ipynb
#BEST 1/16/2020 MODEL 2-INFERENCE (GPU) FULL EXPORTED v2
#      $params: DEBUG = 0
#      $params: delta learn = load_learner($path_model=v2, cpu=False), $TTA params
#               preds, _ = learn_inf.tta(dl=test_dl, n=4, beta=0.25)
#      Score: 0.889  GPU submission time: 15 min Rank: 1434
#      Note: Same model v2 submitted without TTA scored only .877 (see my-kaggle-cassava-1-model_0877a.ipynb)

#latest 1/23/2020 MODEL 2-INFERENCE (GPU) FULL EXPORTED v2bs56
#      $params: DEBUG = 0
#      $params: learn = load_learner($path_model=v2bs56, cpu=False), $TTA params
#               preds, _ = learn_inf.tta(dl=test_dl, n=4, beta=0.25)
#      Score: .884  GPU submission time: 15 min Rank: n/a
#lateset  
#      $params: learn = load_learner($path_model=v2bs56, cpu=False), $TTA params
#               preds, _ = learn_inf.tta(dl=test_dl, n=10, beta=0.75, use_max=True)
#      Score: .881  GPU submission time: 15 min Rank: n/a (no GH)
#      Note: Same model v2bs56 submitted without TTA scored only .881 (see my-kaggle-cassava-1-model_bs56_score0881.ipynb)

#history
#2/7/2021 MODULE 2-INFERENCE MINI EXPORTED v1bs64
#      MINI = 1 no $param
#      $params: learn = load_learner($path_model=v1bs64, cpu=False), $TTA params
#               preds, _ = learn_inf.tta(dl=test_dl, n=4, beta=0.25)
#      Score: .822, .824  GPU submission time: 29sec + submission time ~ 16min Rank: n/a
#      Note: Same model v1bs64 submitted without TTA scored .812 (see mini/my-kaggle-cassava-1-model-mini_score0812.ipynb)

#2/10/2021 MODULE 2-INFERENCE MINI EXPORTED v2bs32
#      MINI = 1 no $param
#      $params: learn = load_learner($path_model=v2bs32, cpu=False), $TTA params
#               preds, _ = learn_inf.tta(dl=test_dl, n=4, beta=0.25)
#      Score: .835, .840  GPU submission time: ?sec + submission time ~ 16min Rank: n/a
#      Note: Same model v2bs32 submitted without TTA scored .822, .825 (see mini/my-kaggle-cassava-1-model-mini_score0822.ipynb)

#Res50
#2/13/2021 MODULE 2-INFERENCE MINI EXPORTED v1 MiniTrain. Res50 std16bs 
#      MINI = 1 no $param
#      $params: learn = load_learner($path_model=v1bs64, cpu=False), $TTA params
#               preds, _ = learn_inf.tta(dl=test_dl, n=4, beta=0.25)
#      Score: .870  GPU submission time: 30sec + submission time ~ 8min Rank: n/a
#      Note: Same model v1 MiniTrain. Res50 std16bs no TTA scored .869 (see mini/my-kaggle-cassava-1-model-mini-res50_score0869.ipynb)

#RESET - BASE FOR FINAL CODE
#Res50
#here 2/14/2021 MODULE 2-INFERENCE FULL EXPORTED MODEL Resnet50 bs=16, DELTA Full, Infer std, no tta
#      Model exported w/MINI = 0 no $param here
#      $params: learn = load_learner($path_model=v full,Res50,bs64, cpu=False)
#               preds, _ = learn.get_preds(dl=test_dl) no tta params
#      Score: .888  GPU submission time: 27sec + submission time, less than 20 min. Rank: n/a
#      Note: Same model v0 1-TrainFull. Res50 std16bs failed, code 137, not scored (see res50/?.ipynb)


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python

import os
#import time #to track performance time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#modeling
from fastai.vision.all import *

# Input data files are available in the read-only "../input/" directory
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#--constants and variables

#set paths
path = Path('../input/cassava-leaf-disease-classification')

#$params
path_model = Path('../input/my-cassava-model-full-res50')

#--functions
def get_x(row): return path/'train_images'/row['image_id']
def get_y(row): return row['label']


# Cassava Disease Classification - Inference
##### Objective: Classify pictures of cassava leaves into 1 of 4 disease categories or healthy.

Export saved model and create a submission.csv file, with predictions on test images.


## 0. Load Prepared Data and Model

- Input(s): export.pkl


In [ ]:
#track time
t0 = time.time()

#check if loaded model exists and load it
if path_model.ls(file_exts='.pkl'):
    print(path_model)
    learn_inf = load_learner(path_model/'export.pkl', cpu=False)
    #learn_inf.to_native_fp32()
else:
    print("No model to export")

## 3. Predict

In [ ]:
##Predict on one file
#view test images - only one
#test_img = get_image_files(path/'test_images')
#Image.open(test_img[0])

#predict one, returns a tuple, get 0th item
#y_hat = learn_inf.predict(test_img[0])
#y_hat[0]

Test dataset

In [ ]:
#template
sample_submission = pd.read_csv(path/'sample_submission.csv')
submission = sample_submission.copy()

In [ ]:
#create test images id
test_img_id = [os.path.join(path/'test_images', x) for x in submission['image_id'].values]

#images in test files
test_img_id

In [ ]:
test_dl = learn_inf.dls.test_dl(test_img_id)
print(len(test_dl))

#predict
#no TTA or TTA params
preds, _ = learn_inf.get_preds(dl=test_dl)
#preds, _ = learn_inf.tta(dl=test_dl, n=4, beta=0.25) #$params
#preds

#submission
submission['label'] = preds.argmax(dim=-1).numpy()
submission.head()

## 4. Submit


In [ ]:
#save to csv
submission.to_csv('submission.csv', index=False)

#total inference time
t1 = time.time()
print ("Inference time (in min)", (time.time() - t0)/60)